In [1]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <complex.h>
// #include "wav_io.h"
#include "./dataset/raw/wanted_beep_files/beep_clear_clip_01_22K.h"

In [2]:
#ifndef _RFFT_H 
#define _RFFT_H
#define M_PI        3.14159265358979323846
#define M_SQRT2     1.41421356237309504880
#endif //end of _RFFT_H

In [3]:
void rfft(float *x, int n, int m);
double parabolic(double* corr, int index);
int find_first_positive(double* d, int length);
int argmax(double* arr, int length);
void correlate(double* signal, int length, double* corr);
double mean(double* signal, int length);
double freq_from_autocorr(double* signal, int length, double fs);
void find(int* condition, int size, int** res, int* res_size);
void detectAudio(double* signal, int sig_len, int sr, double wanted_freq, double magthreshold, double freqthreshold, double (*freq_func)(double*, int));

### utility functions

In [12]:
double mean(double* signal, int length) 
{
    double sum = 0.0;
    for (int i = 0; i < length; i++) 
    {
        sum += signal[i];
    }
    return sum / length;
}

In [13]:
void correlate(double* signal, int length, double* corr) {
    for (int lag = 0; lag < length; lag++) {
        corr[lag] = 0.0;
        for (int i = 0; i < length - lag; i++) {
            corr[lag] += signal[i] * signal[i + lag];
        }
    }
}

In [14]:
int find_first_positive(double* d, int length) 
{
    for (int i = 0; i < length; i++) 
    {
        if (d[i] > 0) 
        {
            return i;
        }
    }
    return -1; // Not found
};

In [15]:
int argmax(double* arr, int length)
{
    int max_index = 0;
    for (int i = 1; i < length; i++) 
    {
        if (arr[i] > arr[max_index]) 
        {
            max_index = i;
        }
    }
    return max_index;
};

In [16]:
double parabolic(double* corr, int index) 
{
    double a = corr[index - 1];
    double b = corr[index];
    double c = corr[index + 1];
    return index + (b - a) / (2 * (b - 2 * a + c));
};

In [17]:
void find(int* condition, int size, int** res, int* res_size) {
    *res_size = 0;
    *res = (int*)malloc(size * sizeof(int));
    
    for (int i = 0; i < size; i++) {
        if (condition[i]) {
            (*res)[(*res_size)++] = i;
        }
    }
    *res = (int*)realloc(*res, (*res_size) * sizeof(int));
}

### fft and rfft implementation

In [8]:
void rfft(float *x, int n, int m)
{
  int j, i, k, is, id;
  int i0, i1, i2, i3, i4, i5, i6, i7, i8;
  int n2, n4, n8;
  float xt, a0, e, a, a3;
  float t1, t2, t3, t4, t5, t6;
  float cc1, ss1, cc3, ss3;
  float *r0;

  /* Digit reverse counter */

  j = 0;
  r0 = x;

  for (i = 0; i < n - 1; i++) {

    if (i < j) {
      xt = x[j];
      x[j] = *r0;
      *r0 = xt;
    }
    r0++;

    k = n >> 1;

    while (k <= j) {
      j = j - k;
      k >>= 1;
    }
    j += k;
  }

  /* Length two butterflies */
  is = 0;
  id = 4;

  while (is < n - 1) {

    for (i0 = is; i0 < n; i0 += id) {
      i1 = i0 + 1;
      a0 = x[i0];
      x[i0] += x[i1];
      x[i1] = a0 - x[i1];
    }

    is = (id << 1) - 2;
    id <<= 2;
  }

  /* L shaped butterflies */
  n2 = 2;
  for (k = 1; k < m; k++) {
    n2 <<= 1;
    n4 = n2 >> 2;
    n8 = n2 >> 3;
    e = (M_PI * 2) / n2;
    is = 0;
    id = n2 << 1;
    while (is < n) {
      for (i = is; i <= n - 1; i += id) {
	i1 = i;
	i2 = i1 + n4;
	i3 = i2 + n4;
	i4 = i3 + n4;
	t1 = x[i4] + x[i3];
	x[i4] -= x[i3];
	x[i3] = x[i1] - t1;
	x[i1] += t1;

	if (n4 != 1) {
	  i1 += n8;
	  i2 += n8;
	  i3 += n8;
	  i4 += n8;
	  t1 = (x[i3] + x[i4]) / M_SQRT2;
	  t2 = (x[i3] - x[i4]) / M_SQRT2;
	  x[i4] = x[i2] - t1;
	  x[i3] = -x[i2] - t1;
	  x[i2] = x[i1] - t2;
	  x[i1] = x[i1] + t2;
	}
      }
      is = (id << 1) - n2;
      id <<= 2;
    }

    for (j = 1; j < n8; j++) {
      a = j * e;
      a3 = 3 * a;
      cc1 = cos(a);
      ss1 = sin(a);
      cc3 = cos(a3);
      ss3 = sin(a3);

      is = 0;
      id = n2 << 1;

      while (is < n) {
	for (i = is; i <= n - 1; i += id) {
	  i1 = i + j;
	  i2 = i1 + n4;
	  i3 = i2 + n4;
	  i4 = i3 + n4;
	  i5 = i + n4 - j;
	  i6 = i5 + n4;
	  i7 = i6 + n4;
	  i8 = i7 + n4;
	  t1 = x[i3] * cc1 + x[i7] * ss1;
	  t2 = x[i7] * cc1 - x[i3] * ss1;
	  t3 = x[i4] * cc3 + x[i8] * ss3;
	  t4 = x[i8] * cc3 - x[i4] * ss3;
	  t5 = t1 + t3;
	  t6 = t2 + t4;
	  t3 = t1 - t3;
	  t4 = t2 - t4;
	  t2 = x[i6] + t6;
	  x[i3] = t6 - x[i6];
	  x[i8] = t2;
	  t2 = x[i2] - t3;
	  x[i7] = -x[i2] - t3;
	  x[i4] = t2;
	  t1 = x[i1] + t5;
	  x[i6] = x[i1] - t5;
	  x[i1] = t1;
	  t1 = x[i5] + t4;
	  x[i5] = x[i5] - t4;
	  x[i2] = t1;
	}
	is = (id << 1) - n2;
	id <<= 2;
      }
    }
  }
}

### frequency estimation algorithms

In [ ]:
void freq_from_fft(double* signal, int N, double fs, double* result) {
    double* windowed = (double*)malloc(N * sizeof(double));
    double* f = (double*)malloc((N/2 + 1) * sizeof(double));
    double* log_f = (double*)malloc((N/2 + 1) * sizeof(double));
    int i_peak, i_interp;

    // Apply Kaiser window
    for (int i = 0; i < N; i++) {
        windowed[i] = signal[i] * kaiser(i, N); // You need to implement kaiser function
    }

    // Compute Fourier transform of windowed signal
    rfft(windowed, f, N); // You need to implement rfft function

    // Find the peak
    i_peak = argmax(f, N/2 + 1); // You need to implement argmax function

    // Interpolate to get a more accurate peak
    parabolic(log_f, i_peak, &i_interp); // You need to implement parabolic function

    // Convert to equivalent frequency
    *result = fs * i_interp / N; // Hz

    free(windowed);
    free(f);
    free(log_f);
}

### main entry

In [5]:
int main(void)
{
    // const char *n_wav2 = "./dataset/raw/neg_test_files/neg_beep_clear_clip_02_22K.wav";
    // WavFile nw2 = wavio_read(n_wav2);
    // float *n_sig2 = nw2.data[0]; // Assuming data is a 2D array(means two channel src wav) and we want the first channel

    // Remember to free resources if necessary
    // wavio_free(nw2);

    return 0;
}

In [18]:
main();

test

In [ ]:
int main() {
    
}